# 1. Load Lib

In [2]:
import re
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np


# 2. [GET DATA]

In [3]:
def get_data(save=False):
    # --- Do the request -----------------------------------------
    req = requests.get('https://www.fundsexplorer.com.br/ranking')
    soup = bs(req.content, 'html.parser')
    # --- Get the data table -------------------------------------
    table = soup.find('table', {'id' : 'table-ranking'})
    # --- Get headers --------------------------------------------
    headers = [th.text for th in table.find_all('th')]
    # --- Get data -----------------------------------------------
    data = []
    trs = table.find('tbody').find_all('tr')
    for tr in trs:
        tds = tr.find_all('td')
        tds_data = []
        for td in tds[2:]:
            added = False
            try:
                tds_data.append(td['data-order'])
                added = True
            except:
                ...
            if(not added):
                try:
                    tds_data.append(td['data-valor'])
                except:
                    tds_data.append(None)
        d = (
            tds[0]['data-index'],
            tds[1].text,
            *tds_data
        )
        data.append(d)
    fiis_data = pd.DataFrame(columns=headers, data=data)
    if(save):
        fiis_data.to_csv('./bases/funds_explorer_database.csv')
    return fiis_data

In [4]:
need_to_get_data = True
fiis_data_loaded = None
if(need_to_get_data):
    fiis_data_loaded = get_data()

---

In [5]:
fiis_data = None
if(fiis_data_loaded is None):
    fiis_data = pd.read_excel('./bases/Funds_Explorer_Data_Base.xlsx')
else:
    fiis_data = fiis_data_loaded

In [6]:
fiis_data

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,...,PatrimônioLíq.,VPA,P/VPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
0,FIVN11,Shoppings,2.15,2274.0,0.0,0.0,0.0,0.0,0.0,0.0,...,65678487.99,6.977698,0.3081245419334571,-9999999999,-9999999999,-9999999999,-9999999999,56.0,-9999999999,1
1,BZLI11,Títulos e Val. Mob.,14.7,44041.0,0.0,0.0,0.0,0.0,0.0,0.0,...,431076817.02,10.703128,1.3734302719728289,-9999999999,-9999999999,-9999999999,-9999999999,-9999999999,-9999999999,0
2,XTED11,Lajes Corporativas,5.98,731.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27305951.24,13.893154,0.43042782078137193,-9999999999,-9999999999,-9999999999,-9999999999,0.0,100.0,1
3,ALMI11,Lajes Corporativas,921.0,172.0,0.0,0.0,0.0,0.0,0.0,0.0,...,249632973.43,2245.365259,0.41017825331909613,-9999999999,-9999999999,-9999999999,-9999999999,64.05,-9999999999,1
4,PABY11,Híbrido,7.45,41.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-9870075.03,-13.014339,-0.5724455156731356,-9999999999,-9999999999,-9999999999,-9999999999,-9999999999,-9999999999,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,BTAL11,Outros,96.0,12597.0,0.83,0.837368845843422,2.52260730823641,5.0673029261007,9.81347550020146,0.84086910274547,...,620294632.17,103.680763,0.9259191119185726,0.800534232179599,0.512968152087723,1.31760886992498,2.10338500034086,-9999999999,-9999999999,7
278,MORC11,Títulos e Val. Mob.,97.66,1698.0,1.5,1.5043626516899,4.43922265582432,8.28610143380024,0.0,1.47974088527477,...,78429486.55,100.558359,0.9711773438944047,1.4916711200508,-0.276558560467444,1.21098721540682,2.32510144153897,-9999999999,-9999999999,0
279,CCRF11,Títulos e Val. Mob.,99.03,3.0,0.74,0.740740740740741,3.20967444463159,6.917533006827,0.0,1.06989148154386,...,159364709.06,101.383491,0.9767862501400745,1.18362465936392,1.36285453644494,2.56261027817348,4.86550749326222,-9999999999,-9999999999,0
280,EQIR11,Títulos e Val. Mob.,96.0,256.0,1.1,1.1340206185567,3.34374482554101,0.0,0.0,1.11458160851367,...,51240359.56,101.970865,0.9414453824629221,1.07873950073877,0.326965532425683,1.40923213951651,2.4568632020173,-9999999999,-9999999999,0


# 3. Transform Data

In [7]:
for col in fiis_data.columns[2:]:
    fiis_data[col] = fiis_data[col].astype('float64')

In [8]:
fiis_data.head()

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,...,PatrimônioLíq.,VPA,P/VPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
0,FIVN11,Shoppings,2.15,2274.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.567849e+07,6.977698,0.308125,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,5.600000e+01,-1.000000e+10,1.0
1,BZLI11,Títulos e Val. Mob.,14.70,44041.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.310768e+08,10.703128,1.373430,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,0.0
2,XTED11,Lajes Corporativas,5.98,731.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.730595e+07,13.893154,0.430428,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,0.000000e+00,1.000000e+02,1.0
3,ALMI11,Lajes Corporativas,921.00,172.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.496330e+08,2245.365259,0.410178,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,6.405000e+01,-1.000000e+10,1.0
4,PABY11,Híbrido,7.45,41.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-9.870075e+06,-13.014339,-0.572446,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,1.0


In [9]:
df = pd.get_dummies(fiis_data, columns=['Setor'])

In [11]:
#color_1hot_df = pd.DataFrame(color_1hot.toarray())

In [12]:
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# # apply on df
# sector_1hot = encoder.fit_transform(fiis_data['Setor'].values.reshape(-1,1))
# sector_1hot_df = pd.DataFrame(color_1hot.toarray())

# 4. Export Data

In [13]:
# for c in df.columns:
#     print(c)

In [14]:
column_name_mapping = {
    'Códigodo fundo' : 'asset',
    'Preço Atual' : 'price',
    'Liquidez Diária' : 'lqdt',
    'Dividendo' : 'divd',
    'DividendYield' : 'dy',
    'DY (3M)Acumulado' : 'dy3ac',
    'DY (6M)Acumulado' : 'dy6ac',
    'DY (12M)Acumulado' : 'dy12ac',
    'DY (3M)Média' : 'dy3m',
    'DY (6M)Média' : 'dy6m',
    'DY (12M)Média' : 'dy12m',
    'DY Ano' : 'dyy',
    'Variação Preço' : 'price_delta',
    'Rentab.Período' : 'profitability',
    'Rentab.Acumulada' : 'profitability_ac',
    'PatrimônioLíq.' : 'liq_patr',
    'VPA' : 'vpa',
    'P/VPA' : 'pvp',
    'DYPatrimonial' : 'dyp',
    'VariaçãoPatrimonial' : 'patr_delta',
#     'Rentab. Patr.no Período' : '',
#     'Rentab. Patr.Acumulada' : '',
#     'VacânciaFísica' : '',
#     'VacânciaFinanceira' : '',
#     'QuantidadeAtivos' : '',
    'Setor_' : 'sector_none',
    'Setor_Hospital' : 'sector_hosp',
    'Setor_Hotel' : 'sector_hotel',
    'Setor_Híbrido' : 'sector_hbd',
    'Setor_Lajes Corporativas' : 'sector_corp',
    'Setor_Logística' : 'sector_log',
    'Setor_Outros' : 'sector_other',
    'Setor_Residencial' : 'sector_resd',
    'Setor_Shoppings' : 'sector_shop',
    'Setor_Títulos e Val. Mob.' : 'sector_title',
}

In [15]:
final_df = df.copy()
for col in final_df.columns:
    if(col in column_name_mapping):
        final_df.rename(columns={col : column_name_mapping[col]}, inplace=True)
    else:
        del final_df[col]

In [16]:
final_df.columns

Index(['asset', 'price', 'lqdt', 'divd', 'dy', 'dy3ac', 'dy6ac', 'dy12ac',
       'dy3m', 'dy6m', 'dy12m', 'dyy', 'price_delta', 'profitability',
       'profitability_ac', 'liq_patr', 'vpa', 'pvp', 'dyp', 'patr_delta',
       'sector_none', 'sector_hosp', 'sector_hotel', 'sector_hbd',
       'sector_corp', 'sector_log', 'sector_other', 'sector_resd',
       'sector_shop', 'sector_title'],
      dtype='object')

In [17]:
df.columns

Index(['Códigodo fundo', 'Preço Atual', 'Liquidez Diária', 'Dividendo',
       'DividendYield', 'DY (3M)Acumulado', 'DY (6M)Acumulado',
       'DY (12M)Acumulado', 'DY (3M)Média', 'DY (6M)Média', 'DY (12M)Média',
       'DY Ano', 'Variação Preço', 'Rentab.Período', 'Rentab.Acumulada',
       'PatrimônioLíq.', 'VPA', 'P/VPA', 'DYPatrimonial',
       'VariaçãoPatrimonial', 'Rentab. Patr.no Período',
       'Rentab. Patr.Acumulada', 'VacânciaFísica', 'VacânciaFinanceira',
       'QuantidadeAtivos', 'Setor_', 'Setor_Hospital', 'Setor_Hotel',
       'Setor_Híbrido', 'Setor_Lajes Corporativas', 'Setor_Logística',
       'Setor_Outros', 'Setor_Residencial', 'Setor_Shoppings',
       'Setor_Títulos e Val. Mob.'],
      dtype='object')

In [18]:
# fiis
fiis = final_df[:100]

In [19]:
print("MODEL: set NP", "'" +  "','".join(fiis.columns[1:]) + "'")

MODEL: set NP 'price','lqdt','divd','dy','dy3ac','dy6ac','dy12ac','dy3m','dy6m','dy12m','dyy','price_delta','profitability','profitability_ac','liq_patr','vpa','pvp','dyp','patr_delta','sector_none','sector_hosp','sector_hotel','sector_hbd','sector_corp','sector_log','sector_other','sector_resd','sector_shop','sector_title'


In [20]:
fdf = final_df.sort_values(by='dy', ascending=False).reset_index(drop=True)

In [21]:
n_assets = 30
fiis = fdf[:n_assets]
filename = 'portifolio_mtx.dat'
sep = '\t'
with open(filename, 'w') as f:
    f.writelines("# PORTIFOLIO ALLOCATION PROBLEM - DATA\n")
    f.writelines(f"param n_assets := {fiis.shape[0]};\n")
    f.writelines(f"param asset_data: {sep.join(fiis.columns[1:])} :=\n")
    for i in range(fiis.shape[0]):
        f.writelines(f"{i+1} {sep.join([str(v) for v in fiis.loc[i].values[1:]])} #{fiis.loc[i].values[0]}\n")
    f.writelines(f";\n")

---

In [22]:
# TESTES:

In [23]:
fdf = final_df.sort_values(by='dy', ascending=False).reset_index(drop=True)

In [24]:
fdf

,asset,price,lqdt,divd,dy,dy3ac,dy6ac,dy12ac,dy3m,dy6m,...,sector_none,sector_hosp,sector_hotel,sector_hbd,sector_corp,sector_log,sector_other,sector_resd,sector_shop,sector_title
0,KINP11,12.86,2854.0,0.370736,2.896379e+00,7.326577e+00,0.000000e+00,0.000000e+00,2.442192e+00,0.000000e+00,...,0,0,0,0,0,0,0,1,0,0
1,JPPA11,105.34,4909.0,2.000000,1.924928e+00,5.215091e+00,9.390879e+00,1.578337e+01,1.738364e+00,1.565146e+00,...,0,0,0,0,0,0,0,0,0,1
2,RZAK11,96.71,14442.0,1.800000,1.837672e+00,4.884843e+00,9.036357e+00,1.485705e+01,1.628281e+00,1.506059e+00,...,0,0,0,0,0,0,0,0,0,1
3,MBRF11,518.00,91.0,10.000000,1.818182e+00,4.563266e+00,8.498132e+00,1.454691e+01,1.521089e+00,1.416355e+00,...,0,0,0,0,0,0,1,0,0,0
4,URPR11,114.20,49775.0,2.010000,1.807066e+00,4.938155e+00,9.136861e+00,1.858542e+01,1.646052e+00,1.522810e+00,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,ORPD11,93.00,3.0,1.379310,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,...,0,0,0,0,0,0,0,0,0,1
278,LFTT11,97.45,255.0,0.747633,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,...,0,0,0,0,0,0,0,1,0,0
279,FMOF11,62.00,6.0,0.640000,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,...,0,0,0,0,1,0,0,0,0,0
280,CJCT11,79.50,2.0,0.000000,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,-1.000000e+10,...,0,0,0,0,1,0,0,0,0,0
